# Introduction to TensorFlow
This notebook will focus on the fundamentals of the TensorFlow platform (Python) in a follow-along style tutorial. 
## 1. Handle Imports
First we need to import TensorFlow. It is common practice to import it as "tf" and to also check the version you are using. 

In [1]:
import tensorflow as tf
print(tf.__version__)

2.10.0
